In [6]:
from edward.models import MultivariateNormalTriL, Normal, Poisson
from edward.util import rbf
import numpy as np
import pandas as pd
import edward as ed
import tensorflow as tf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import GPy
GPy.plotting.change_plotting_library('plotly')
from IPython.display import display
init_notebook_mode(connected=True)

In [7]:
counts = pd.read_csv('data/zip_counts.csv')
counts = counts[(counts['LAT'] > 25) & (counts['LAT'] < 50) & (counts['LNG'] > -125) & (counts['LNG'] < -67) &(counts['population'] > 0)]

In [8]:
step = 0.25
to_bin = lambda x: np.floor(x / step) * step
counts["latbin"] = counts.LAT.map(to_bin)
counts["lonbin"] = counts.LNG.map(to_bin)
grouped = pd.DataFrame(counts.groupby(['latbin', 'lonbin', 'committee_name'], as_index = False)['count', 'population'].sum())

In [9]:
clinton = grouped[grouped['committee_name'] == 'HILLARY FOR AMERICA']
iplot([go.Scatter3d(x = clinton['lonbin'], y = clinton['latbin'], z=clinton['count'], mode = 'markers', marker=dict(size = 2,))])

## Edward

In [10]:
from edward.models import MultivariateNormalTriL, Normal, Poisson
from edward.util import rbf

In [11]:
N = clinton.shape[0]
D = 2

x_ph = tf.placeholder(tf.float32, (N,D))  # inputs to Gaussian Process
K = rbf(x_ph, lengthscale=10., variance = 1.) + tf.diag(tf.ones(N)*1e-3)
f = MultivariateNormalTriL(loc=tf.ones((N))*np.log(clinton['count'].mean()), scale_tril=tf.contrib.distributions.matrix_diag_transform(K))
y = Normal(loc=tf.exp(f), scale = 1.0)

In [12]:
qf = Normal(loc=tf.Variable(tf.random_normal([N])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([N]))))

inference = ed.KLqp({f: qf}, data={x_ph: np.array(clinton[['latbin','lonbin']]), y: np.array(clinton['count'])})
inference.run(n_iter=500)

500/500 [100%] ██████████████████████████████ Elapsed: 189s | Loss: 31250270208.000

In [ ]:
counts = pd.read_csv('data/zip_counts.csv')

In [ ]:
counts